In [ ]:
#Noun+Verb LemSpacy LDA GBM: 0.8071395816658974

In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim import corpora
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')
sp = spacy.load('en_core_web_lg')

categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(categories=categories, shuffle=True)
data = newsgroup.data
target = newsgroup.target

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def lemmatizeSpacy(input):
    doc = sp(input)
    #Tokenizers divide strings into lists of substrings
    output = ' '.join([w.lemma_ for w in doc])
    return output

In [4]:
def NVExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_ in ["NOUN","VERB"]:
            output+=w.text+" "
    output = output[:-1]
    return output

In [5]:
def train(x_train, x_test, y_train, y_test, n):
    clf = GradientBoostingClassifier(n_estimators=n)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    return f1_score(y_test, y_pred, average='weighted')

In [6]:
def textProcessing(unprocessedTexts, function=None):
    if function is None:
        return unprocessedTexts
    return [function(text) for text in unprocessedTexts]

In [7]:
dataExt = textProcessing(data,NVExt)
dataProc = textProcessing(dataExt,lemmatizeSpacy)
tokenized_documents = [simple_preprocess(text) for text in dataProc]
dictionary = corpora.Dictionary(tokenized_documents)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [ ]:
alphaArr = ['auto',0.0001,0.01,0.1,0.5,0.7,1.2,1.8,2]
betaArr = ['auto',0.0001,0.01,0.1,0.5,0.7,1.2,1.8,2]
numArr = [3,7,20,40,90,140,200,275,400]
mArr = [10,30,80,100,180,225,300,500]
list = []
for num in numArr:
    for m in mArr:
        listAB = []
        for alp in alphaArr:
            for beta in betaArr:
                length = num
                lda = LdaModel(bow_corpus, num_topics=num, id2word=dictionary, alpha=alp, eta=beta)
                dataVecPre = lda[bow_corpus]
                len_data = len(dataVecPre)
                dataVec = [0 for o in range(len_data)]
                for o in range(len_data):
                    curr = []
                    a = dataVecPre[o]
                    lenCurr = len(a)
                    if length!=lenCurr:
                        curr = [0 for u in range(length)]
                        c = 0
                        for u in range(length):
                            if c>=lenCurr:
                                curr[u] = 0
                            else:
                                if a[c][0]!=u:
                                    curr[u] = 0
                                else:
                                    curr[u] = a[c][1]
                                    c+=1
                        dataVec[o] = curr
                    else:
                        curr = [element[1] for element in a]
                        dataVec[o] = curr
                dataVec = np.array(dataVec)
                X_train, X_test, y_train, y_test = train_test_split(dataVec, target, test_size=0.33)
                f1 = train(X_train, X_test, y_train, y_test, m)
                list.append([f1,alp,beta,num,m])
                listAB.append([f1,alp,beta,num,m])
        maxListAB = [w[0] for w in listAB]
        maximumAB = max(maxListAB)
        print(maximumAB)
        print(listAB[maxListAB.index(maximumAB)])
    print("--------------------------------------------------------")
maxList = [w[0] for w in list]
maximum = max(maxList)
print(maximum)
print(list[maxList.index(maximum)])

0.6014781773933384
[0.6014781773933384, 0.7, 0.1, 3, 10]
0.598341102807229
[0.598341102807229, 1.8, 'auto', 3, 30]
0.6179589561577139
[0.6179589561577139, 'auto', 0.1, 3, 80]
0.5880683266310885
[0.5880683266310885, 0.7, 0.7, 3, 100]
0.6379370122646092
[0.6379370122646092, 2, 0.0001, 3, 180]
0.5596866000622504
[0.5596866000622504, 1.2, 2, 3, 225]
0.5879647344217522
[0.5879647344217522, 1.2, 1.8, 3, 300]
0.5875626309514766
[0.5875626309514766, 1.8, 1.2, 3, 500]
--------------------------------------------------------
0.743661104763838
[0.743661104763838, 2, 1.8, 7, 10]
0.6822266321551598
[0.6822266321551598, 1.2, 1.2, 7, 30]
0.7497983645539509
[0.7497983645539509, 1.8, 'auto', 7, 80]
0.7190393680729891
[0.7190393680729891, 1.8, 1.8, 7, 100]
0.7116703233037398
[0.7116703233037398, 1.8, 2, 7, 180]
0.6939745760798393
[0.6939745760798393, 0.7, 0.0001, 7, 225]
0.7104754089174767
[0.7104754089174767, 1.2, 2, 7, 300]
0.7484435885674375
[0.7484435885674375, 1.8, 2, 7, 500]
----------------------

In [ ]:
!Увидеть распределение по темам+скинуть трансформеры